# COGS 118A - Project Checkpoint

# Names

- Cristian Antonio-Hernandez
- Rahul Puranam
- Ricardo Sedano
- Jason Shao

# Abstract 
Our goal is to develop an effective email spam detection system. We will be using a dataset consisting of labeled emails, where each email is classified as either spam or non-spam. The data represents various features extracted from the emails, such as the presence of certain keywords, email header information, and textual content. We will employ machine learning algorithms to train a model on this labeled data to accurately classify incoming emails as spam or non-spam. The performance of the system will be measured by evaluating its ability to correctly classify a new set of emails, using metrics such as accuracy, precision, recall, and F1-score. Our objective is to achieve high accuracy and minimize false positives (legitimate emails classified as spam) and false negatives (spam emails classified as legitimate), ensuring an efficient and reliable spam detection system.

# Background

The field of email spam detection has been extensively researched, with significant prior work paving the way for advancements in this area. Various techniques and approaches have been explored to tackle the problem of email classification, aiming to accurately distinguish between spam and non-spam emails.

Researchers have employed machine learning algorithms, such as support vector machines (SVMs), naive Bayes classifiers, and ensemble methods, to build effective spam detection models<a name="solanki"></a>[<sup>[1]</sup>](#solanki)<a name="dada"></a>[<sup>[2]</sup>](#dada). These models utilize features extracted from email content, such as keyword presence, textual analysis, and email header information, to make predictions on the classification of incoming emails.

Feature engineering has been a crucial aspect of this research, as it involves selecting and extracting relevant features that contribute to the identification of spam emails<a name="dada"></a>[<sup>[2]</sup>](#dada). Additionally, researchers have explored the use of natural language processing techniques to analyze the text of emails, including techniques like text tokenization, stemming, and TF-IDF weighting<a name="sahami"></a>[<sup>[3]</sup>](#sahami)<a name="yang"></a>[<sup>[4]</sup>](#yang).

# Problem Statement

Our problem is how to predict if an email is spam or not spam based on email content, using features such as keyword presence, textual analysis, and email header information. We aim to optimize the loss function so that we are able to minimize the false positives and false negatives. Some ML-relevant potential solutions are Support Vector Machines, Decision Trees, Naive Bayes classifiers, and ensemble methods such as Random Forest.

# Data

The ideal dataset for our project would be made up of a couple thousand observations, at least 2000 observations, with a varying amount of variables depending on individual email data. For exmaple, and observation should consist of email length, time and date sent, sender email address, letter capitalization ratio, and average amount of special characters used per sentence. 

Because our project deals with a categorical classificiation problem, the critical variables could be represented in a regression model. Pertaining to the scenario, critical values would hypothetically be tied to special character and keyword usage, and sender's email header and address, as outliers and trends in these varibales these would signal a higher likelihood of spam.

Additional to obtaining the data, we hope to not handle data more than needed in order to preserve integrity in the data, and reduce bias thoughout. In order to illustrate an effective classification model and to help midigate overfitting, L2 regularization, pruning, and cross-validation can be valuable processes needed to sighly transform data, and balance the bias-variance tradeoff.

Upon researching, email data from the internet may not be as robust as we anticipate. UCI Machine Learning Repository's 'Spambase Data Set' is a promising dataset, yet contains missing values.<a name="uci"></a>[<sup>[5]</sup>](#uci) Kaggle and other resources also have datasets, yet are more focused on SMS (Text) Messaging Spam. We will look at these datasets to further solidify and view parallels between email and SMS spam which we can later look to for strenghtening the classfication problem.

https://huggingface.co/datasets/sms_spam

----------

UPDATED FROM PROPOSAL!

You should have obtained and cleaned (if necessary) data you will use for this project.

Please give the following infomration for each dataset you are using
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc you have done should be demonstrated here!


# Proposed Solution

We are going to use support vector machines to obtain the hyperplane to place the decision boundary and to create a margin of each feature using the training dataset, in which we would need to calculate the optimum pair of w and b that yields the minimum loss. Or in other words, finding a margin that is wide as possible with the fewest errors made on the decision boundary. We will be implementing gradient descent to find the minimum loss and possibly using kernels to transform our data into data that is linearly separable in higher dimensions. To test our solution we will be measuring the accuracy of our classifier on the testing dataset and using the F1 score to tell us about our performance on false positives and false negatives. 

# Evaluation Metrics

One evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model is the F1-score (F1-score = $\frac{2 * (\text{Precision} * \text{Recall})}{(\text{Precision} + \text{Recall})}$), assessing both precision and recall, as precision represents the ability of the model to correctly identify spam emails among the emails it classifies as spam, and recall measures the ability of the model to identify all the actual spam emails.

# Preliminary results

NEW SECTION!

Please show any preliminary results you have managed to obtain.

Examples would include:
- Analyzing the suitability of a dataset or alogrithm for prediction/solving your problem 
- Performing feature selection or hand-designing features from the raw data. Describe the features available/created and/or show the code for selection/creation
- Showing the performance of a base model/hyper-parameter setting.  Solve the task with one "default" algorithm and characterize the performance level of that base model.
- Learning curves or validation curves for a particular model
- Tables/graphs showing the performance of different models/hyper-parameters



# Ethics & Privacy

The most obvious ethical concern that should be addressed is having access to emails, of which for the average email recipient, may include personal information, like residential address, close contacts, and calendar events. Going back a step, people may not even want their email address to get leaked, so having this data be used may raise concern. Continuing on the email message contents, a detection service would have to scan a large portion of the email, or each line of text to properly make a decision. Having an algorithm read every email automatically aould also cause concern.

To address these issues, a step toward the correct solution would be to automatically not store and get rid of all text that the model takes in. From a user standpoint, it may also be helpful to give permission to the model to only filter email spam from senders outside of the recipients most popular contacts, or turn off or deactivative the spam detection completely if the user feels unsure if the detection service is too invasive or unethical.

# Team Expectations 

* *Team members will actively contribute to group discussions and decision-making processes.*
* *Any questions or concerns should be brought up in a timely manner, so that they can be addressed as soon as possible.*
* *All work will be submitted on time and meet the project's standards and guidelines.*
* *Regular meeting times should be scheduled in advance to check progress and ensure the project is on task.*

# Project Timeline Proposal

UPDATE THE PROPOSAL TIMELINE ACCORDING TO WHAT HAS ACTUALLY HAPPENED AND HOW IT HAS EFFECTED YOUR FUTURE PLANS

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 5/16  |  3 PM |  Brainstorm topics/questions (all)  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 5/17  | Before 11:59 PM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 5/14  | 6 PM  | Import & Wrangle Data, do some EDA | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 5/31  | Before 11:59 PM  | Finalize wrangling/EDA; Begin programming for project | Discuss/edit project code; Complete project |
| 6/10  | 12 PM  | Complete analysis; Draft results/conclusion/discussion | Discuss/edit full project |
| 6/14  | Before 11:59 PM  | NA | Turn in Final Project  |

# Footnotes
<a name="solanki"></a>1.[^](#solanki): Solanki, Rohit Kumar, et al. Spam Filtering Using Hybrid Local-Global Naive Bayes Classifier, 28 Sept. 2015, www.semanticscholar.org/paper/Spam-filtering-using-hybrid-local-global-Naive-Solanki-Verma/978a7972210e2d771dac6f92e17594100ea1a8a6. <br> 
<a name="dada"></a>2.[^](#dada): Dada, Emmanuel Gbenga, et al. “Machine Learning for Email Spam Filtering: Review, Approaches and Open Research Problems.” Heliyon, vol. 5, no. 6, June 2019, p. e01802, https://doi.org/10.1016/j.heliyon.2019.e01802. <br>
<a name="sahami"></a>3.[^](#sahami): Sahami, Mehran, et al. A B a Yesian Approach to Filtering Junk E-Mail. http://erichorvitz.com/ftp/junkfilter.pdf. <br>
<a name="yang"></a>4.[^](#yang): Yang, Yiming, and Xin Liu. “A Re-Examination of Text Categorization Methods.” Proceedings of the 22nd Annual International ACM SIGIR Conference on Research and Development in Information Retrieval - SIGIR ’99, 1999, https://doi.org/10.1145/312624.312647. <br>
<a name="uci"></a>4.[^](#uci): “Spambase Data Set” UCI Machine Learning Repository Archives, https://archive.ics.uci.edu/ml/datasets/spambase. <br>